In [1]:
from pathlib import Path
import pandas as pd

INTERIM_DIR = Path("./data/interim")
PROCESSED_DIR = Path("./data/processed")
NO_MEANING_DIR = Path("./data/no_meaning")

In [2]:
%load_ext autoreload
%autoreload 2
from prepareData import sliding_window
import train
#import verify_hf_models

/home/ubuntu/My_LogLLM/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


n_epochs_1: 1
n_epochs_2_1: 1
n_epochs_2_2: 1
n_epochs_3: 2
dataset_name: windows_security
batch_size: 16
micro_batch_size: 4
lr_1: 0.0005
lr_2_1: 0.0005
lr_2_2: 5e-05
lr_3: 5e-05
max_content_len: 100
max_seq_len: 128
min_less_portion: 0.3
device: cuda:0


## データ準備

In [4]:
sliding_window.main(
    data_dir = NO_MEANING_DIR/"T1105",
    log_filename = "security2",  # {log_filename}_structured.csv を参照することに注意！
    output_dir = PROCESSED_DIR / "T1105_full",
    group_type = "fixedSize",
)

Auto log_format: <Version>,<Computer>,<Execution_ThreadID>,<Channel>,<Content>,<Provider_Name>,<Correlation_RelatedActivityID>,<Keywords>,<Opcode>,<Correlation_ActivityID>,<Execution_ProcessID>,<Security_UserID>,<Task>,<Level>,<Provider_Guid>,<TimeCreated_SystemTime>,<EventRecordID>,<EventID>,<project>
window_size: 100; step_size: 100
1027687
Start grouping.
Train dataset info:
max session length: 100; mean session length: 99.99379712965215

number of anomalous sessions: 0; number of normal sessions: 8222; number of total sessions: 8222

Test dataset info:
max session length: 100; mean session length: 99.96984435797665

number of anomalous sessions: 11; number of normal sessions: 2045; number of total sessions: 2056



## モデル動かす

In [3]:
train.main(data_path=PROCESSED_DIR/"T1105_full"/"train.csv")

dataset: data/processed/T1105_full/train.csv
Number of normal samples in original dataset: 8222
Number of anomalous samples in original dataset: 0


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

### 下準備
- 作業単純化のため、一旦Drainで正規化済みのデータを使用する
- このためのカラム名調整

In [ ]:
data = pd.read_csv(INTERIM_DIR/"T1105/security2_structured.csv")
data = data.rename(columns={
    "Content": "Raw_Content",
    "EventTemplate": "Content"
})
data.to_csv(INTERIM_DIR/"T1105/security2_structured.csv")

In [6]:
data = pd.read_csv(NO_MEANING_DIR/"T1105/security2_structured.csv")
data = data.rename(columns={
    "Content": "Raw_Content",
    "EventTemplate": "Content"
})
data.to_csv(NO_MEANING_DIR/"T1105/security2_structured.csv")